In [1]:
from preprocessing import df, pd

In [2]:
df

,No.,Time,Source IP,Destination IP,Protocol,Length,Source Port,Destination Port,Resolved Source Mac,Source Mac Address,TCP Flags
0,1,0.000000,192.168.0.107,192.168.0.255,UDP,305,59199.0,54915.0,IntelCor,a0:88:69:e0:2d:d3,None
1,2,0.102253,192.168.0.107,192.168.0.255,UDP,305,54915.0,54915.0,IntelCor,a0:88:69:e0:2d:d3,None
2,3,0.244179,192.168.0.100,192.168.0.2,DNS,81,61098.0,53.0,Apple,8c:85:90:56:69:c0,None
3,4,0.305297,IntelCor_6c:2f:50,Broadcast,ARP,42,NaN,NaN,IntelCor,cc:3d:82:6c:2f:50,None
4,5,0.308302,192.168.0.2,192.168.0.100,ICMP,109,61098.0,53.0,D-LinkIn,3c:1e:04:12:86:d8,None
...,...,...,...,...,...,...,...,...,...,...,...
117328,117329,71.852720,192.168.0.100,192.168.0.121,TCP,66,80.0,64484.0,Apple,8c:85:90:56:69:c0,"(SYN, ACK)"
117329,117330,71.852737,192.168.0.100,192.168.0.121,TCP,66,80.0,64463.0,Apple,8c:85:90:56:69:c0,"(SYN, ACK)"
117330,117331,71.852776,192.168.0.100,192.168.0.121,TCP,66,80.0,64464.0,Apple,8c:85:90:56:69:c0,"(SYN, ACK)"
117331,117332,71.866342,192.168.0.100,192.168.0.104,TCP,460,80.0,58082.0,Apple,8c:85:90:56:69:c0,"(FIN, PSH, ACK)"


### Source Ports involved in the attack


In [3]:
attack_entries = df.loc[df['Destination IP'] == "192.168.0.100"]
attack_entries = attack_entries.loc[attack_entries['Protocol'] == 'TCP']
attack_entries

,No.,Time,Source IP,Destination IP,Protocol,Length,Source Port,Destination Port,Resolved Source Mac,Source Mac Address,TCP Flags
128,129,19.975082,192.168.0.112,192.168.0.100,TCP,66,33600.0,80.0,IntelCor,ac:ed:5c:b9:d8:a9,SYN
130,131,19.977576,192.168.0.112,192.168.0.100,TCP,54,33600.0,80.0,IntelCor,ac:ed:5c:b9:d8:a9,ACK
132,133,19.994433,192.168.0.112,192.168.0.100,TCP,80,33600.0,80.0,IntelCor,ac:ed:5c:b9:d8:a9,"(PSH, ACK)"
136,137,19.997342,192.168.0.112,192.168.0.100,TCP,54,33600.0,80.0,IntelCor,ac:ed:5c:b9:d8:a9,ACK
137,138,20.011530,192.168.0.112,192.168.0.100,TCP,80,33600.0,80.0,IntelCor,ac:ed:5c:b9:d8:a9,"(PSH, ACK)"
...,...,...,...,...,...,...,...,...,...,...,...
117313,117314,71.852104,192.168.0.121,192.168.0.100,TCP,66,64483.0,80.0,IntelCor,7c:5c:f8:89:d8:c5,SYN
117314,117315,71.852105,192.168.0.121,192.168.0.100,TCP,66,64462.0,80.0,IntelCor,7c:5c:f8:89:d8:c5,SYN
117315,117316,71.852106,192.168.0.121,192.168.0.100,TCP,66,64484.0,80.0,IntelCor,7c:5c:f8:89:d8:c5,SYN
117316,117317,71.852107,192.168.0.121,192.168.0.100,TCP,66,64463.0,80.0,IntelCor,7c:5c:f8:89:d8:c5,SYN
